***UIT_CS115*** - 
***FINAL PROJECT***

_**TOPIC** : POLICY GRADIENT FOR REINFORCEMENT LEARNING_


The Actor-Critic algorithm is a type of **Reinforcement Learning** algorithm that combines aspects of both Policy-based methods (Actor) and Value-based methods (Critic). This hybrid approach is designed to address the limitations of each method when used individually.

In the actor-critic framework, an agent (the "actor") learns a policy to make decisions, and a value function (the "Critic") evaluates the actions taken by the Actor.

Simultaneously, the critic evaluates these actions by estimating their value or quality. This dual role allows the method to strike a balance between exploration and exploitation, leveraging the strengths of both policy and value functions.



In [7]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical


In [8]:
env = gym.make('CartPole-v1', render_mode='human')
env.reset()
# torch.manual_seed(args.seed)

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


In [9]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


In [10]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


In [11]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + 0.99 * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


In [ ]:
def main():
    running_reward = 0

    # run infinitely many episodes
    for i_episode in count(1):

        # reset environment and episode reward
        state, _ = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 100001):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _, _ = env.step(action)
            env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = ep_reward +  running_reward

        # perform backprop
        finish_episode()

        # log results
        log_interval = 1
        if i_episode % log_interval == 0:
            print('Episode {}\tReward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward / i_episode))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))


if __name__ == '__main__':
    main()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 1	Reward: 13.00	Average reward: 13.00
Episode 2	Reward: 10.00	Average reward: 11.50
Episode 3	Reward: 10.00	Average reward: 11.00
Episode 4	Reward: 10.00	Average reward: 10.75
Episode 5	Reward: 10.00	Average reward: 10.60
Episode 6	Reward: 8.00	Average reward: 10.17
Episode 7	Reward: 10.00	Average reward: 10.14
Episode 8	Reward: 9.00	Average reward: 10.00
Episode 9	Reward: 9.00	Average reward: 9.89
Episode 10	Reward: 9.00	Average reward: 9.80
Episode 11	Reward: 9.00	Average reward: 9.73
Episode 12	Reward: 10.00	Average reward: 9.75
Episode 13	Reward: 10.00	Average reward: 9.77
Episode 14	Reward: 8.00	Average reward: 9.64
Episode 15	Reward: 11.00	Average reward: 9.73
Episode 16	Reward: 9.00	Average reward: 9.69
Episode 17	Reward: 10.00	Average reward: 9.71
Episode 18	Reward: 12.00	Average reward: 9.83
Episode 19	Reward: 9.00	Average reward: 9.79
Episode 20	Reward: 10.00	Average reward: 9.80
Episode 21	Reward: 9.00	Average reward: 9.76
Episode 22	Reward: 10.00	Average reward: 9.7

KeyboardInterrupt: 

: 